In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
%matplotlib inline
import seaborn as sns
from scipy import stats
from datetime import datetime 
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import KNNImputer
import category_encoders as ce
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures

In [19]:
data=pd.read_csv("model_data.csv")
street=pd.read_csv("boston.csv",names=["stree_name","latitude","longitude"])

In [20]:
data.head()

,Unnamed: 0,distance,surge_multiplier,Lyft_Type,Uber_Type,cab_type_Uber,price
0,0,0.02,1.0,0.0,3.0,1,6.5
1,1,0.02,1.0,0.0,6.0,1,15.0
2,2,0.02,1.0,0.0,7.0,1,27.5
3,3,0.02,1.0,0.0,2.0,1,6.5
4,4,0.02,1.0,0.0,4.0,1,8.5


In [21]:
data=data.iloc[:,1:]

In [22]:
data.head()

,distance,surge_multiplier,Lyft_Type,Uber_Type,cab_type_Uber,price
0,0.02,1.0,0.0,3.0,1,6.5
1,0.02,1.0,0.0,6.0,1,15.0
2,0.02,1.0,0.0,7.0,1,27.5
3,0.02,1.0,0.0,2.0,1,6.5
4,0.02,1.0,0.0,4.0,1,8.5


In [23]:
data.distance.value_counts()

2.66    7225
2.32    7171
2.84    6777
1.41    6198
1.25    5879
        ... 
4.99       4
4.85       4
3.84       3
4.89       3
5.06       3
Name: distance, Length: 511, dtype: int64

In [24]:
poly_features = PolynomialFeatures(degree=6)

In [25]:
X=data.loc[:,['distance', 'surge_multiplier', 'Lyft_Type', 'Uber_Type', 'cab_type_Uber']]
Y=data['price']

In [26]:
X_train_poly = poly_features.fit_transform(X)

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X_train_poly, Y, test_size = 0.40,shuffle=True,random_state=10)

In [29]:
import time

poly_model = linear_model.LinearRegression()
start_time = time.time()
poly_model.fit(X_train, y_train)
print("--- %s seconds ---" % (time.time() - start_time))

--- 67.37051796913147 seconds ---


In [30]:
# predicting on test data-set
prediction= poly_model.predict(X_test)
print("--- %s seconds ---" % (time.time() - start_time))

--- 68.64051938056946 seconds ---


In [31]:
# print('Co-efficient of linear regression',poly_model.coef_)
print('Intercept of linear regression model',poly_model.intercept_)
print('Mean Square Error', metrics.mean_squared_error(y_test, prediction))
print("r2_score:",metrics.r2_score(prediction,y_test)*100,"%")

Intercept of linear regression model -3591878.2547301683
Mean Square Error 3.2033302454761072
r2_score: 95.94656193936568 %


In [14]:
# import pickle
# pickle.dump(poly_model,open('modell.pkl','wb'))

In [32]:
from math import radians, cos, sin, asin, sqrt
def distance(lat1, lat2, lon1, lon2):
     
    # The math module contains a function named
    # radians which converts from degrees to radians.
    lon1 = radians(lon1)
    lon2 = radians(lon2)
    lat1 = radians(lat1)
    lat2 = radians(lat2)
      
    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
 
    c = 2 * asin(sqrt(a))
    
    # Radius of earth in kilometers. Use 3956 for miles
    r = 6371
      
    # calculate the result
    return(c * r)
     

In [33]:
street.head()

,stree_name,latitude,longitude
0,3rd Place 212,42.337923,-71.048038
1,4th Street Place 2127,42.334637,-71.042955
2,A Street 2210,42.345284,-71.052449
3,A Street 2127,42.345280,-71.052454
4,Aberdeen Street 2215,42.346413,-71.103843


In [34]:
id_name_dict={
    "55c66225-fbe7-4fd5-9072-eab1ece5e23e":"UberX",
    "8cf7e821-f0d3-49c6-8eba-e679c0ebcf6a":"Taxi",
    "997acbb5-e102-41e1-b155-9df7de0a73f2":"UberPool",
    "6d318bcc-22a3-4af6-bddd-b409bfce1546":"Black SUV",
    "9a0e7b09-b92b-4c41-9779-2ad22b4d779d":"WAV",
    "6f72dfc5-27f1-42e8-84db-ccc7a75f6969":"UberXL",
    "6c84fd89-3f11-4782-9b50-97c468b19529":"Black",
    "lyft_plus":"Lyft XL",
    "lyft_lux":"Lux Black",
    "lyft":"Lyft",
    "lyft_luxsuv":"Lux Black XL",
    "lyft_premier":"Lux",
    "lyft_line":"Shared"
}

In [35]:
uber_names=['Black SUV','Black','WAV','UberXL','UberX','Taxi','UberPool']
lyft_names=['Shared', 'Lyft', 'Lyft XL', 'Lux', 'Lux Black', 'Lux Black XL']  

In [36]:
def message(col_name):
    print(col_name+" has wrong value")
    print("1:Drop row")
    print("2:replace with another value")

In [55]:
# distance	cab_type	time_stamp	destination	source	surge_multiplier	id	product_id	name	price
def check_dataset(datasetname):
    # important columns[distance - cab_type - lyft_type - uber_type]
    data=pd.read_csv(datasetname)
    for i in range(data.shape[0]):
#         distance imputation
        if (data.iloc[i,0] <=0.0)or (data.iloc[i,0]==np.nan):
            if ((data.iloc[i,1]!=np.nan or data.iloc[i,1]!="") and (data.iloc[i,2]!=np.nan or data.iloc[i,2]!="") ):
                    lat1=street[street.stree_name==data.iloc[i,1]].latitude
                    lat2=street[street.stree_name==data.iloc[i,2]].latitude
                    longi=street[street.stree_name==data.iloc[i,1]].longitude
                    logi2=street[street.stree_name==data.iloc[i,2]].longitude
                    
                    data.iloc[i,0]=distance(lat1,lat2,longi,longi2)
            
            else:
                    data.iloc[i,0]=data.distance.mean()
                    
        
        if ((data.iloc[i,7] ==np.nan)or (data.iloc[i,7]=="")):
                    data.iloc[i,7]=id_name_dic[data.iloc[i,6]]
        
        if ((data.iloc[i,1] =="")or (data.iloc[i,1]==np.nan)):
            if data.iloc[i,7] in uber_names:
                    data.iloc[i,1]="Uber"
            elif data.iloc[i,7] in lyft_names:
                    data.iloc[i,1]="Lyft"
        
        if (data.iloc[i,5] <=0.0)or (data.iloc[i,5]==np.nan):
                    data.iloc[i,5]=1.0
       
    return data
        
        

In [189]:
csv_name="taxi-tas-regression-test.csv"
data=pd.read_csv(csv_name)
# data = data.assign(source_destination = data.destination+data.source) 

In [190]:
data.name.unique()

array(['Lux Black XL', 'UberPool', 'Lyft XL', 'Lyft', 'UberX', 'Lux',
       'WAV', 'Lux Black', 'Black SUV', 'Black', 'UberXL', 'Shared'],
      dtype=object)

In [191]:
data=data[data.name!='Taxi']

In [192]:
data.name.unique()

array(['Lux Black XL', 'UberPool', 'Lyft XL', 'Lyft', 'UberX', 'Lux',
       'WAV', 'Lux Black', 'Black SUV', 'Black', 'UberXL', 'Shared'],
      dtype=object)

In [158]:
# #  taxi price indiction from uberx
# source_dest_un=data[data['name']=='UberX']['source_destination'].unique()
# print(len(source_dest_un))


# index=data[data['name']=='Taxi'].index.values.tolist()

# uberx_data=data[data['name']=='UberX']

# for i in index:    
#    dest_taxi=data.iloc[i,0]
#    so_dest=data.iloc[i,-1]
#    data.iloc[i,7]=uberx_data[(uberx_data['source_destination']==so_dest) & (uberx_data['distance']==dest_taxi)]['price'].mean()


72


In [193]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18498 entries, 0 to 18497
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   distance          18498 non-null  float64
 1   cab_type          18498 non-null  object 
 2   time_stamp        18498 non-null  float64
 3   destination       18498 non-null  object 
 4   source            18498 non-null  object 
 5   surge_multiplier  18498 non-null  float64
 6   id                18498 non-null  object 
 7   product_id        18498 non-null  object 
 8   name              18498 non-null  object 
 9   price             18498 non-null  float64
dtypes: float64(4), object(6)
memory usage: 1.6+ MB


In [194]:
data=check_dataset(csv_name)

In [195]:
data.head()

,distance,cab_type,time_stamp,destination,source,surge_multiplier,id,product_id,name,price
0,2.14,Lyft,1.543230e+12,South Station,North Station,1.0,d17c8f8f-5018-4dc1-b86b-0e67849e5f57,lyft_luxsuv,Lux Black XL,27.5
1,1.08,Uber,1.543280e+12,North End,North Station,1.0,4258d0c6-e1e5-45fb-bcee-d2a02aa5e6b0,997acbb5-e102-41e1-b155-9df7de0a73f2,UberPool,6.5
2,0.57,Lyft,1.543630e+12,North Station,Haymarket Square,1.0,a0a9b20e-d5ed-4eab-b2a5-d6ebd5eb3214,lyft_plus,Lyft XL,9.0
3,1.08,Lyft,1.543590e+12,Northeastern University,Back Bay,1.0,11f9ed79-2c2c-465a-8501-da97893ebcfc,lyft,Lyft,7.0
4,3.20,Uber,1.543240e+12,Financial District,Northeastern University,1.0,be3939d1-a5ee-43cc-a8ab-6f0da31594c5,55c66225-fbe7-4fd5-9072-eab1ece5e23e,UberX,11.0


In [196]:

# data=check_dataset(csv_name)
# data=data[:,1:]
data.drop(['time_stamp','destination','source','id','product_id'],axis=1,inplace=True)
data.cab_type=data.cab_type.str.lower()
data.name=data.name.str.lower()
data= pd.get_dummies(data, columns=["cab_type"])
data.drop(['cab_type_lyft'],axis=1,inplace=True)

lyft={'lux black xl':6,'lux black':5,'lux':4,'lyft xl':3,'lyft':2,'shared':1}
lyft_map=[{'col':'Lyft_Type','mapping':lyft}]

data['Lyft_Type']=data['name']
encodeing= ce.OrdinalEncoder(cols=['Lyft_Type'],return_df=True,mapping=lyft_map)
data=encodeing.fit_transform(data)
data.loc[data.Lyft_Type <1, 'Lyft_Type'] = 0

uber={'black suv':6,'black':5,'wav':4,'uberxl':3,'uberx':2,'uberpool':1}
uber_map=[{'col':'Uber_Type','mapping':uber}]
data['Uber_Type']=data['name']
encodeing= ce.OrdinalEncoder(cols=['Uber_Type'],return_df=True,mapping=uber_map)
data=encodeing.fit_transform(data)
data.loc[data.Uber_Type <1, 'Uber_Type'] = 0

data.drop(['name'], axis = 1,inplace=True)
X_input=data.loc[:,['distance', 'surge_multiplier', 'Lyft_Type', 'Uber_Type', 'cab_type_uber']]
Y_input=data['price']

In [197]:
import pickle


In [198]:
X_input.head()

,distance,surge_multiplier,Lyft_Type,Uber_Type,cab_type_uber
0,2.14,1.0,6.0,0.0,0
1,1.08,1.0,0.0,1.0,1
2,0.57,1.0,3.0,0.0,0
3,1.08,1.0,2.0,0.0,0
4,3.20,1.0,0.0,2.0,1


In [199]:
X_input.shape

(18498, 5)

In [200]:
poly_features = PolynomialFeatures(degree=6)

In [201]:
pickle_mod=pickle.load(open("poly_model",'rb'))

In [202]:
pred=pickle_mod.predict(poly_features.fit_transform(X_input))

In [203]:
print('Mean Square Error', metrics.mean_squared_error(pred,Y_input))
print("r2_score:",metrics.r2_score(pred,Y_input)*100,"%")

Mean Square Error 105.58614822541173
r2_score: 48.48112757241749 %


In [147]:
csv_name="taxi-tas-classification-test.csv"

In [148]:
data=check_dataset(csv_name)

In [149]:
data.head()

,distance,cab_type,time_stamp,destination,source,surge_multiplier,id,product_id,name,RideCategory
0,2.14,Lyft,1.543230e+12,South Station,North Station,1.0,d17c8f8f-5018-4dc1-b86b-0e67849e5f57,lyft_luxsuv,Lux Black XL,expensive
1,1.08,Uber,1.543280e+12,North End,North Station,1.0,4258d0c6-e1e5-45fb-bcee-d2a02aa5e6b0,997acbb5-e102-41e1-b155-9df7de0a73f2,UberPool,cheap
2,0.57,Lyft,1.543630e+12,North Station,Haymarket Square,1.0,a0a9b20e-d5ed-4eab-b2a5-d6ebd5eb3214,lyft_plus,Lyft XL,cheap
3,1.08,Lyft,1.543590e+12,Northeastern University,Back Bay,1.0,11f9ed79-2c2c-465a-8501-da97893ebcfc,lyft,Lyft,cheap
4,3.20,Uber,1.543240e+12,Financial District,Northeastern University,1.0,be3939d1-a5ee-43cc-a8ab-6f0da31594c5,55c66225-fbe7-4fd5-9072-eab1ece5e23e,UberX,moderate


In [150]:
data.drop(['time_stamp','destination','source','id','product_id'],axis=1,inplace=True)
data.cab_type=data.cab_type.str.lower()
data.name=data.name.str.lower()
data= pd.get_dummies(data, columns=["cab_type"])
# data.drop(['cab_type_lyft'],axis=1,inplace=True)

lyft={'lux black xl':6,'lux black':5,'lux':4,'lyft xl':3,'lyft':2,'shared':1}
lyft_map=[{'col':'Lyft_Type','mapping':lyft}]

data['Lyft_Type']=data['name']
encodeing= ce.OrdinalEncoder(cols=['Lyft_Type'],return_df=True,mapping=lyft_map)
data=encodeing.fit_transform(data)
data.loc[data.Lyft_Type <1, 'Lyft_Type'] = 0

uber={'black suv':7,'black':6,'wav':5,'uberxl':4,'uberx':3,'taxi':2,'uberpool':1}
uber_map=[{'col':'Uber_Type','mapping':uber}]
data['Uber_Type']=data['name']
encodeing= ce.OrdinalEncoder(cols=['Uber_Type'],return_df=True,mapping=uber_map)
data=encodeing.fit_transform(data)
data.loc[data.Uber_Type <1, 'Uber_Type'] = 0

data.drop(['name'], axis = 1,inplace=True)
Ride={'very expensive':4,'expensive':3,'moderate':2,'cheap':1,'unknown':0}
Ride_Category_map=[{'col':'Ride_Category','mapping':Ride}]
data['Ride_Category']=data['RideCategory']
encodeing= ce.OrdinalEncoder(cols=['Ride_Category'],return_df=True,mapping=Ride_Category_map)
data=encodeing.fit_transform(data)
data.drop(['RideCategory'],axis=1,inplace=True)

X_input=data.loc[:,['distance', 'surge_multiplier','cab_type_lyft','cab_type_uber', 'Lyft_Type', 'Uber_Type']]
Y_input=data['Ride_Category']

In [151]:
print(data.head())

   distance  surge_multiplier  cab_type_lyft  cab_type_uber  Lyft_Type  \
0      2.14               1.0              1              0        6.0   
1      1.08               1.0              0              1        0.0   
2      0.57               1.0              1              0        3.0   
3      1.08               1.0              1              0        2.0   
4      3.20               1.0              0              1        0.0   

   Uber_Type  Ride_Category  
0        0.0              3  
1        1.0              1  
2        0.0              1  
3        0.0              1  
4        3.0              2  


In [152]:
pickle_mod=pickle.load(open("randomforest",'rb'))

In [153]:
pred=pickle_mod.predict(X_input)

In [154]:
print("Accuracy:",metrics.accuracy_score(Y_input, pred))

Accuracy: 0.8799329657260244
